In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from googlemaps.client import Client
import time
import random

In [ ]:
def getListItem(list):
    res = ''
    if list is not None:
        for r in list:
            res += r + ','
        res = res[:-1]
    return res
def parseRecommend(recommend):
    recommends = ''
    for i in range(len(recommend)):
        try:
            recommend[i][1][11]
            recommends += recommend[i][1][11] + ';' + getListItem(recommend[i][1][13]) + ';'
        except:
            recommends += ''
    return recommends
def getRecommend(place_id):
    apiKey = '' #your api key
    gmaps = Client(key = apiKey)
    placeId = place_id
    try:
        info = gmaps.place(place_id = placeId, language='zh-TW')
        url = info.get('result').get('url')
        time.sleep(random.uniform(0,2))
        headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        scripts = soup.head.find_all('script')
        for script in scripts:
            if 'window.APP_INITIALIZATION_STATE' in script.contents[0]:
                script_text = script.contents[0]
        ds_text = script_text[script_text.index('window.APP_INITIALIZATION_STATE')+32:script_text.index(';window.APP_FLAGS')]
        ds_text = ds_text.replace(")]}\'", '')
        ds = json.loads(ds_text)
        ds2 = json.loads(ds[3][6])
        try:
            recommend = ds2[6][99][0][0][1]
        except TypeError:
            recommend = []
    except:
        recommend = []
    res = parseRecommend(recommend) if len(recommend) != 0 else ''
    return res

In [ ]:
raw_data = pd.read_excel('C:\\Users\\daniel.huang\\Desktop\\唐宮蒙古烤肉涮羊肉餐廳.xlsx')
raw_data = raw_data[(raw_data['評分'] == 5) & (raw_data['評論內容'].notna())].head(500)

In [ ]:
uid_list = raw_data['uid'].to_list()

In [ ]:
review_list = []
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
for item in uid_list:
    target_uid = item
    url = 'https://www.google.com/maps/contrib/'+target_uid+'/reviews'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser") 
    scripts = soup.head.find_all('script')
    script_text = ''
    for script in scripts:
        if 'window.APP_INITIALIZATION_STATE' in script.contents[0]:
            script_text = script.contents[0]
    ds_text = script_text[script_text.index('window.APP_INITIALIZATION_STATE')+32:script_text.index(';window.APP_FLAGS')]
    ds_text = ds_text.replace(")]}\'", '')
    ds = json.loads(ds_text)
    ds2 = json.loads(ds[3][9])    
    #UID
    try:
        uid = ds2[16][14]
    except TypeError:
        uid = ''
    #名稱
    try:
        name = ds2[16][0]
    except TypeError:
        name = ''
    #積分
    try:
        score = ds2[16][8][1][0]
    except TypeError:
        score = ''
    #等級
    try:
        rank = ds2[16][8][1][1]
    except TypeError:
        rank = ''
    #評分數
    try:
        number_of_score = ds2[13][0][0]
    except TypeError:
        number_of_score = ''
    #評論數
    try:
        number_of_reviews = ds2[13][1][0]
    except TypeError:
        number_of_reviews = ''
    #評論(前10筆)
    try:
        review = ds2[24][0]
    except TypeError:
        review = ''
    except IndexError:
        review = ''        
    print('UID:' + str(uid) + ' 名稱:' + str(name))    
    for i in range(len(review)):
        place_id = ''
        place_name = ''
        place_url = ''
        post_time = ''
        rank = ''
        tag = ''
        full_address = ''
        district = ''
        road = ''
        content = ''
        recommend = ''
        place_id = review[i][1][18]
        place_name = review[i][1][2]
        place_url = 'https://www.google.com' + review[i][1][17][0]
        post_time = review[i][0][1]
        rank = review[i][0][4]
        tag = getListItem(review[i][1][4])
        full_address = review[i][1][3]
        district = review[i][1][26]
        road = review[i][1][20]
        content = review[i][0][3] if review[i][0][3] is not None else ''
        recommend = getRecommend(place_id)
        review_list.append([uid, name, score, rank, number_of_score, number_of_reviews, place_id, place_name, place_url, post_time, rank, tag, full_address, district, road, content, recommend])

In [ ]:
columns = ['uid', '評論者名稱', '積分', '等級', '評分數', '評論數', 'place_id', '商家名稱', '商家url', '評論時間', '評價', '商家類型', '地址', '行政區', '路名', '評論內容', '推薦商家']
DF = pd.DataFrame(review_list, columns=columns)
DF.to_excel('C:\\Users\\daniel.huang\\Desktop\\唐宮蒙古烤肉涮羊肉餐廳_userdata.xlsx', index=False, engine='xlsxwriter')

In [ ]:
print('UID:' + str(uid))
print('名稱:' + str(name))
print('積分:' + str(score))
print('等級:' + str(rank))
print('評分數:' + str(number_of_score))
print('評論數:' + str(number_of_reviews))